# Working with Spark

AP-HP's clinical data warehouse uses Spark to distribute computations on a cluster. This example notebook shows how one can leverage PySpark to apply an NLP pipeline on a Spark DataFrame.

The Spark "connector" simply wraps the pipeline into a UDF and distributes it on a cluster.

### Getting clinical notes

This section supposes you have an PySpark `sql` object ready to use.
If not, it can be created e.g. via

```python
from pyspark.sql.session import SparkSession

spark = SparkSession.builder.getOrCreate()
sql = spark.sql
```

Now let us query some notes

In [2]:
DB_NAME = 'edsomop_prod_a'
TABLE_NAME = 'note'
N = 1000

notes = sql(
    f"""
    SELECT
        person_id,
        visit_occurrence_id,
        note_id,
        note_text
    FROM
        {DB_NAME}.{TABLE_NAME}
    LIMIT {N}"""
)

### Defining an NLP pipeline

Now, as you would do normally using EDS-NLP, let us define the pipeline whe want to apply on out notes.  
In this example, we will construct a *dummy* `matcher`:

In [4]:
# Importating Spacy and all EDS-NLP pipes
import spacy
import edsnlp.components

In [5]:
# Creates the Spacy instance
nlp = spacy.blank('fr')

# Normalisation of accents, case and other special characters
nlp.add_pipe('normalizer')
# Detecting end of lines
nlp.add_pipe('sentences')

# Extraction of named entities
nlp.add_pipe(
    'matcher',
    config=dict(
        terms=dict(respiratoire=[
            'difficultes respiratoires',
            'asthmatique',
            'toux',
        ]),
        regex=dict(
            covid=r'(?i)(?:infection\sau\s)?(covid[\s\-]?19|corona[\s\-]?virus)',
            traitement=r'(?i)traitements?|medicaments?',
            respiratoire="respiratoires",
        ),
        attr='NORM',
    ),
)

# Qualification of matched entities
nlp.add_pipe('negation')
nlp.add_pipe('hypothesis')
nlp.add_pipe('family')
nlp.add_pipe('rspeech')

### Applying the pipeline

As shown above, we have defined a matcher which extracts entities, and added some qualifiers that add attributes to those entities.  
Let us mention the used qualifiers here:

In [6]:
qualifiers = ['negated', 'hypothesis', 'reported_speech', 'family']

Finally, to apply the pipeline to the PySpark DataFrame, two options are available

In [17]:
import pyspark.sql.functions as F
from edsnlp.connectors.spark import udf_factory, apply_nlp

#### 1. The `udf_factory` function

This function allows us to define a matcher:

In [14]:
matcher = udf_factory(
    nlp=nlp,
    qualifiers=qualifiers,
)

Now what's left to do is to apply this *matcher* on the DataFrame

In [18]:
# Apply the matcher
note_nlp = notes.withColumn("matches", matcher(notes.note_text))

# Formatting the output into separate columns
note_nlp = note_nlp.withColumn("matches", F.explode(note_nlp.matches))

# Selection the columns of interest
note_nlp = note_nlp.select("note_id", "matches.*")

That's it, you now have a DataFrame containing one row per extracted entity, with the following columns:

In [19]:
note_nlp.columns

['note_id',
 'lexical_variant',
 'label',
 'discarded',
 'start',
 'end',
 'negated',
 'hypothesis',
 'reported_speech',
 'family']

#### 2. The `apply_nlp` function

In [21]:
note_nlp = apply_nlp(notes, nlp)

In [22]:
note_nlp.columns

['note_nlp_id',
 'note_id',
 'lexical_variant',
 'label',
 'discarded',
 'start',
 'end',
 'note_nlp_datetime']

#### 3. Go deeper

Out of the box, these two methods allows you to use relatively simple `spacy` pipes.
However, you might want to
- Extract spans stored as `SpanGroups` in the `doc.spans` dictionnary
- Extract specific user-defined extensions (e.g. the `score_method` extension of the **SOFA** pipeline)
For this, the two methods `udf_factory` and `apply_nlp` exposes 2 keyword arguments

**The `additionnal_spans` argument**:  
This argument can be either a string or a list of string. It tells the method to also retrieve `Spans` 
from the `doc.spans` dictionnary, using the provided key(s).

**The `additionnal_extensions` argument**:  
This argument should be a list of 2-Tuples. Each tuple is of the following structure:
- The first element is a string designing the name of the extension, omitting the leading `_`.
- The second element is a `pyspark.sql.types` type giving the type of the extension.